The Goal here (pun intended) is to design a prediction system which can accurately predict if the home team will win or not. We will use the final dataset got by our earlier "Scraping and Cleaning" Notebook build our prediction model on.

In [1]:
# Import the necessary libraries.
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from IPython.display import display

%matplotlib inline

In [2]:
# Read data and drop redundant column.
data = pd.read_csv('../input/football-results-and-betting-odds-data-of-epl/final_dataset_with_odds.csv')
test = pd.read_csv('../input/football-results-and-betting-odds-data-of-epl/with_odds_test.csv')


In [3]:
col1 = ['FTR','HTGS', 'ATGS', 'HTGC', 'ATGC', 'HomeTeamLP', 'AwayTeamLP',
'HTWinStreak3', 'HTWinStreak5', 'HTLossStreak3', 'HTLossStreak5',
'ATWinStreak3', 'ATWinStreak5', 'ATLossStreak3', 'ATLossStreak5',
'HTFormPts','ATFormPts',
'HTGD', 'ATGD', 'DiffPts', 'DiffFormPts', 'DiffLP']

col2 = ['FTR','HTGS', 'ATGS', 'HTGC', 'ATGC', 'HomeTeamLP', 'AwayTeamLP',
'HTWinStreak3', 'HTWinStreak5', 'HTLossStreak3', 'HTLossStreak5',
'ATWinStreak3', 'ATWinStreak5', 'ATLossStreak3', 'ATLossStreak5',
'HTFormPts','ATFormPts',
'HTGD', 'ATGD', 'DiffPts', 'DiffFormPts', 'DiffLP','B365H','B365D','B365A','IWH','IWD','IWA','LBH','LBD','LBA','WHH','WHD','WHA']

col3 = ['FTR','B365H','B365D','B365A','IWH','IWD','IWA','LBH','LBD','LBA','WHH','WHD','WHA']

col4 = ['FTR','HTGS', 'ATGS', 'HTGC', 'ATGC', 'HomeTeamLP', 'AwayTeamLP',
'HTGD', 'ATGD', 'DiffPts', 'DiffFormPts', 'DiffLP',
'HTFormPts','ATFormPts',        
'B365H','B365D','B365A','IWH','IWD','IWA','LBH','LBD','LBA','WHH','WHD','WHA']

In [4]:
# Remove first 3 matchweeks
#data = data[data.MW > 3]
'''
data.drop([
    'Unnamed: 0','HomeTeam', 'AwayTeam_x', 'Date', 'AwayTeam_y',
    'MW',
    #'HTGD', 'ATGD',
    'HTFormPtsStr', 'ATFormPtsStr', 
    'FTHG', 'FTAG',
    #'HTGS', 'ATGS', 'HTGC', 'ATGC',
    #'HomeTeamLP', 'AwayTeamLP',
    #'DiffPts','DiffFormPts','DiffLP',
    #'HTFormPtsStr', 'ATFormPtsStr',
    'HTP', 'ATP',
    'HTFormPts','ATFormPts',
    #'B365H','B365D','B365A','IWH','IWD','IWA','LBH','LBD','LBA','WHH','WHD','WHA'
    'HM1', 'HM2', 'HM3', 'AM1', 'AM2', 'AM3','HM4','HM5','AM4','AM5'
    #,'HTLossStreak5','ATLossStreak5','HTWinStreak5','ATWinStreak5'
    #,'HTWinStreak3','HTLossStreak3','ATWinStreak3','ATLossStreak3'
    
],1, inplace=True)
'''
data = data[col4]

# Preview data.
display(data.head())

,FTR,HTGS,ATGS,HTGC,ATGC,HomeTeamLP,AwayTeamLP,HTGD,ATGD,DiffPts,...,B365A,IWH,IWD,IWA,LBH,LBD,LBA,WHH,WHD,WHA
0,NH,0,0,0,0,10.0,17.0,0.0,0.0,0.0,...,4.333,1.8,3.1,3.8,1.615,3.25,5.00,1.66,3.3,4.50
1,NH,0,0,0,0,14.0,6.0,0.0,0.0,0.0,...,2.200,2.9,3.0,2.2,2.800,3.20,2.20,2.75,3.1,2.30
2,NH,0,0,0,0,15.0,9.0,0.0,0.0,0.0,...,2.750,2.3,3.0,2.7,2.250,3.20,2.75,2.30,3.1,2.75
3,H,0,0,0,0,13.0,16.0,0.0,0.0,0.0,...,4.333,1.8,3.1,3.8,1.833,3.20,3.75,1.72,3.2,4.33
4,H,0,0,0,0,5.0,18.0,0.0,0.0,0.0,...,4.500,1.7,3.2,4.2,1.615,3.50,4.50,1.66,3.3,4.50


## Data Exploration

In [5]:
# Total number of students.
n_matches = data.shape[0]

# Calculate number of features.
n_features = data.shape[1] - 1

# Calculate matches won by home team.
n_homewins = len(data[data.FTR == 'H'])

# Calculate win rate for home team.
win_rate = (float(n_homewins) / (n_matches)) * 100

# Print the results
print("Total number of matches: {}".format(n_matches))
print("Number of features: {}".format(n_features))
print("Number of matches won by home team: {}".format(n_homewins))
print("Win rate of home team: {:.2f}%".format(win_rate))

Total number of matches: 6000
Number of features: 25
Number of matches won by home team: 2787
Win rate of home team: 46.45%


# Visualising distribution of data
from pandas.plotting import scatter_matrix
odds = ['B365H','B365D','B365A','IWH','IWD','IWA','LBH','LBD','LBA','WHH','WHD','WHA']


scatter_matrix(data[odds[::-1]], figsize=(10,10))

scatter_matrix(data[['B365H','B365D','B365A','IWH','IWD','IWA','LBH','LBD','LBA','WHH','WHD','WHA']], figsize=(10,10))

## Preparing the Data

In [6]:
data.columns

Index(['FTR', 'HTGS', 'ATGS', 'HTGC', 'ATGC', 'HomeTeamLP', 'AwayTeamLP',
       'HTGD', 'ATGD', 'DiffPts', 'DiffFormPts', 'DiffLP', 'HTFormPts',
       'ATFormPts', 'B365H', 'B365D', 'B365A', 'IWH', 'IWD', 'IWA', 'LBH',
       'LBD', 'LBA', 'WHH', 'WHD', 'WHA'],
      dtype='object')

In [7]:
# Separate into feature set and target variable
X_all = data.drop(['FTR'],1)
y_all = data['FTR']
'''
# Standardising the data.
from sklearn.preprocessing import scale
# all
cols = [['HTGS', 'ATGS', 'HTGC', 'ATGC','DiffLP'
         ,'HomeTeamLP', 'AwayTeamLP'
         #,'B365H','B365D','B365A','IWH','IWD','IWA','LBH','LBD','LBA','WHH','WHD','WHA'
        ]]
# odds
#cols = [['B365H','B365D','B365A','IWH','IWD','IWA','LBH','LBD','LBA','WHH','WHD','WHA'
#        ]]
for col in cols:
    X_all[col] = scale(X_all[col])
'''
    


"\n# Standardising the data.\nfrom sklearn.preprocessing import scale\n# all\ncols = [['HTGS', 'ATGS', 'HTGC', 'ATGC','DiffLP'\n         ,'HomeTeamLP', 'AwayTeamLP'\n         #,'B365H','B365D','B365A','IWH','IWD','IWA','LBH','LBD','LBA','WHH','WHD','WHA'\n        ]]\n# odds\n#cols = [['B365H','B365D','B365A','IWH','IWD','IWA','LBH','LBD','LBA','WHH','WHD','WHA'\n#        ]]\nfor col in cols:\n    X_all[col] = scale(X_all[col])\n"

X_all.HM1 = X_all.HM1.astype('str')
X_all.HM2 = X_all.HM2.astype('str')
X_all.HM3 = X_all.HM3.astype('str')
X_all.AM1 = X_all.AM1.astype('str')
X_all.AM2 = X_all.AM2.astype('str')
X_all.AM3 = X_all.AM3.astype('str')

def preprocess_features(X):
    ''' Preprocesses the football data and converts catagorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix = col)
                    
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print("Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns)))

In [8]:
# Show the feature information by printing the first five rows
print("\nFeature values:")
display(X_all.head())


Feature values:


,HTGS,ATGS,HTGC,ATGC,HomeTeamLP,AwayTeamLP,HTGD,ATGD,DiffPts,DiffFormPts,...,B365A,IWH,IWD,IWA,LBH,LBD,LBA,WHH,WHD,WHA
0,0,0,0,0,10.0,17.0,0.0,0.0,0.0,0.0,...,4.333,1.8,3.1,3.8,1.615,3.25,5.00,1.66,3.3,4.50
1,0,0,0,0,14.0,6.0,0.0,0.0,0.0,0.0,...,2.200,2.9,3.0,2.2,2.800,3.20,2.20,2.75,3.1,2.30
2,0,0,0,0,15.0,9.0,0.0,0.0,0.0,0.0,...,2.750,2.3,3.0,2.7,2.250,3.20,2.75,2.30,3.1,2.75
3,0,0,0,0,13.0,16.0,0.0,0.0,0.0,0.0,...,4.333,1.8,3.1,3.8,1.833,3.20,3.75,1.72,3.2,4.33
4,0,0,0,0,5.0,18.0,0.0,0.0,0.0,0.0,...,4.500,1.7,3.2,4.2,1.615,3.50,4.50,1.66,3.3,4.50


In [9]:
X_all.info

<bound method DataFrame.info of       HTGS  ATGS  HTGC  ATGC  HomeTeamLP  AwayTeamLP      HTGD      ATGD  \
0        0     0     0     0        10.0        17.0  0.000000  0.000000   
1        0     0     0     0        14.0         6.0  0.000000  0.000000   
2        0     0     0     0        15.0         9.0  0.000000  0.000000   
3        0     0     0     0        13.0        16.0  0.000000  0.000000   
4        0     0     0     0         5.0        18.0  0.000000  0.000000   
...    ...   ...   ...   ...         ...         ...       ...       ...   
5995    20    15    23    30        14.0        18.0 -0.176471 -0.882353   
5996    17    25    19    26        18.0        12.0 -0.117647 -0.058824   
5997    19    33    22    13        15.0         5.0 -0.176471  1.176471   
5998    34    28     6    26         4.0         2.0  1.647059  0.117647   
5999    13    35    30    20        17.0         6.0 -1.000000  0.882353   

       DiffPts  DiffFormPts  ...  B365A   IWH  IWD   IW

In [10]:
from sklearn.model_selection import train_test_split

# Shuffle and split the dataset into training and testing set.
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, 
                                                    test_size = 1000,
                                                    random_state = 20,
                                                    stratify = y_all)

## Training and Evaluating Models

In [11]:
from time import time 
from sklearn.metrics import f1_score

def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print("Trained model in {:.4f} seconds".format(end - start))

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    
    end = time()
    # Print and return results
    print("Made predictions in {:.4f} seconds.".format(end - start))
    
    return f1_score(target, y_pred, pos_label='H'), sum(target == y_pred) / float(len(y_pred))


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print("Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train)))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    f1, acc = predict_labels(clf, X_train, y_train)
    print(f1, acc)
    print("F1 score and accuracy score for training set: {:.4f} , {:.4f}.".format(f1 , acc))
    
    f1, acc = predict_labels(clf, X_test, y_test)
    print("F1 score and accuracy score for test set: {:.4f} , {:.4f}.".format(f1 , acc))

In [12]:
# TODO: Initialize the three models (XGBoost is initialized later)
clf_A = LogisticRegression(random_state = 2021)
clf_B = SVC(random_state = 2021, kernel='rbf')
clf_C = xgb.XGBClassifier(seed = 2021)

train_predict(clf_A, X_train, y_train, X_test, y_test)
print('')
train_predict(clf_B, X_train, y_train, X_test, y_test)
print('')
train_predict(clf_C, X_train, y_train, X_test, y_test)
print('')

Training a LogisticRegression using a training set size of 5000. . .


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Trained model in 0.2120 seconds
Made predictions in 0.0037 seconds.
0.6052076002814919 0.6634
F1 score and accuracy score for training set: 0.6052 , 0.6634.
Made predictions in 0.0031 seconds.
F1 score and accuracy score for test set: 0.6175 , 0.6680.

Training a SVC using a training set size of 5000. . .
Trained model in 1.7286 seconds
Made predictions in 1.0441 seconds.
0.5689914701455094 0.6564
F1 score and accuracy score for training set: 0.5690 , 0.6564.
Made predictions in 0.2120 seconds.
F1 score and accuracy score for test set: 0.5764 , 0.6620.

Training a XGBClassifier using a training set size of 5000. . .
Trained model in 0.4467 seconds
Made predictions in 0.0177 seconds.
0.9736095965103598 0.9758
F1 score and accuracy score for training set: 0.9736 , 0.9758.
Made predictions in 0.0051 seconds.
F1 score and accuracy score for test set: 0.5967 , 0.6310.



**Clearly XGBoost seems like the best model as it has the highest F1 score and accuracy score on the test set.**

# Tuning the parameters of XGBoost.

In [13]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer


# TODO: Create the parameters list you wish to tune
parameters = { 'learning_rate' : [0.01],
               'n_estimators' : [200],
               'max_depth': [5],
               'min_child_weight': [3],
               'gamma':[0.4],
               'subsample' : [0.8],
               'colsample_bytree' : [0.8],
               'scale_pos_weight' : [1],
               'reg_alpha':[1e-5]
              
             }  

# TODO: Initialize the classifier
clf = xgb.XGBClassifier(seed=2,early_stopping_rounds=10)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score,pos_label='H')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf,
                        scoring=f1_scorer,
                        param_grid=parameters,
                        cv=5)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train,y_train,verbose=True)

# Get the estimator
clf = grid_obj.best_estimator_
print(clf)

# Report the final F1 score for training and testing after parameter tuning
f1, acc = predict_labels(clf, X_train, y_train)
print("F1 score and accuracy score for training set: {:.4f} , {:.4f}.".format(f1 , acc))

f1, acc = predict_labels(clf, X_test, y_test)
print("F1 score and accuracy score for test set: {:.4f} , {:.4f}.".format(f1 , acc))

[10:07:20] WARNING: ../src/learner.cc:516: 
Parameters: { early_stopping_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[10:07:21] WARNING: ../src/learner.cc:516: 
Parameters: { early_stopping_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[10:07:21] WARNING: ../src/learner.cc:516: 
Parameters: { early_stopping_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if y

In [14]:
# Remove first 3 matchweeks
#test = test[test.MW > 3]
'''
test.drop([
    'Unnamed: 0','HomeTeam', 'AwayTeam_x', 'Date', 'AwayTeam_y',
    'MW',
    'HTGD', 'ATGD',
    'HTFormPtsStr', 'ATFormPtsStr', 
    'FTHG', 'FTAG',
    'HTGS', 'ATGS', 'HTGC', 'ATGC',
    'HomeTeamLP', 'AwayTeamLP',
    'DiffPts','DiffFormPts','DiffLP',
    'HTFormPtsStr', 'ATFormPtsStr',
    'HTP', 'ATP',
    'HTFormPts','ATFormPts',
    #'B365H','B365D','B365A','IWH','IWD','IWA','LBH','LBD','LBA','WHH','WHD','WHA'
    'HM1', 'HM2', 'HM3', 'AM1', 'AM2', 'AM3','HM4','HM5','AM4','AM5'
    ,'HTLossStreak5','ATLossStreak5','HTWinStreak5','ATWinStreak5'
    ,'HTWinStreak3','HTLossStreak3','ATWinStreak3','ATLossStreak3'
],1, inplace=True)
'''
test = test[col4]
# Preview data.
display(test.head())

# Separate into feature set and target variable
test_x = test.drop(['FTR'],1)
test_y = test['FTR']
'''
# Standardising the data.
from sklearn.preprocessing import scale
# all
cols = [['HTGS', 'ATGS', 'HTGC', 'ATGC','DiffLP'
         ,'HomeTeamLP', 'AwayTeamLP'
         #,'B365H','B365D','B365A','IWH','IWD','IWA','LBH','LBD','LBA','WHH','WHD','WHA'
        ]]
# odds
#cols = [['B365H','B365D','B365A','IWH','IWD','IWA','LBH','LBD','LBA','WHH','WHD','WHA'
#        ]]
for col in cols:
    test_x[col] = scale(test_x[col])
    '''

,FTR,HTGS,ATGS,HTGC,ATGC,HomeTeamLP,AwayTeamLP,HTGD,ATGD,DiffPts,...,B365A,IWH,IWD,IWA,LBH,LBD,LBA,WHH,WHD,WHA
0,NH,19,37,19,7,18.0,4.0,0.000000,1.666667,-1.111111,...,1.57,6.00,4.00,1.57,6.85,4.04,1.59,6.50,3.90,1.55
1,H,37,15,23,33,6.0,7.0,0.777778,-1.000000,1.222222,...,15.00,1.25,6.10,11.00,1.23,6.40,14.96,1.22,6.00,15.00
2,H,25,20,28,24,12.0,15.0,-0.166667,-0.222222,0.111111,...,4.33,1.85,3.60,4.30,1.95,3.55,4.40,1.91,3.50,4.20
3,NH,17,29,33,29,18.0,2.0,-0.888889,0.000000,-0.666667,...,1.57,5.80,4.05,1.57,6.27,4.10,1.62,6.00,4.00,1.57
4,NH,35,21,14,21,5.0,9.0,1.166667,0.000000,0.833333,...,11.00,1.35,5.30,8.00,1.33,5.42,11.37,1.30,5.25,11.00


"\n# Standardising the data.\nfrom sklearn.preprocessing import scale\n# all\ncols = [['HTGS', 'ATGS', 'HTGC', 'ATGC','DiffLP'\n         ,'HomeTeamLP', 'AwayTeamLP'\n         #,'B365H','B365D','B365A','IWH','IWD','IWA','LBH','LBD','LBA','WHH','WHD','WHA'\n        ]]\n# odds\n#cols = [['B365H','B365D','B365A','IWH','IWD','IWA','LBH','LBD','LBA','WHH','WHD','WHA'\n#        ]]\nfor col in cols:\n    test_x[col] = scale(test_x[col])\n    "

# Fitting the model on the whole dataset for future predictions.

In [15]:
from pprint import pprint


# TODO: Create the parameters list you wish to tune
parameters = { 'learning_rate' : [0.01],
               'n_estimators' : [5000],
               'max_depth': [9],
               'min_child_weight': [5],
               'gamma':[0.2],
               'subsample':[0.8],
               'colsample_bytree':[0.8],
               'scale_pos_weight' : [1],
               'reg_alpha':[1e-2]
             }  

# TODO: Initialize the classifier
clf = xgb.XGBClassifier(seed=2)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score,pos_label='H')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf,
                        scoring=f1_scorer,
                        param_grid=parameters,
                        cv=5)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_all,y_all)

# Get the estimator
clf = grid_obj.best_estimator_
print(clf)

# Report the final F1 score for training and testing after parameter tuning
f1, acc = predict_labels(clf, X_all, y_all)
print("F1 score and accuracy score for training set: {:.4f} , {:.4f}.".format(f1 , acc))
f1, acc = predict_labels(clf, test_x, test_y)
print("F1 score and accuracy score for test set: {:.4f} , {:.4f}.".format(f1 , acc))

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0.2, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.01, max_delta_step=0, max_depth=9,
              min_child_weight=5, missing=nan, monotone_constraints='()',
              n_estimators=5000, n_jobs=0, num_parallel_tree=1, random_state=2,
              reg_alpha=0.01, reg_lambda=1, scale_pos_weight=1, seed=2,
              subsample=0.8, tree_method='exact', validate_parameters=1,
              verbosity=None)
Made predictions in 1.2787 seconds.
F1 score and accuracy score for training set: 0.9996 , 0.9997.
Made predictions in 0.1306 seconds.
F1 score and accuracy score for test set: 0.5768 , 0.6169.


In [16]:
y_pred = clf.predict(test_x)
y_pred

array(['NH', 'NH', 'NH', 'NH', 'H', 'NH', 'H', 'H', 'H', 'H', 'NH', 'H',
       'H', 'NH', 'NH', 'H', 'H', 'H', 'NH', 'NH', 'NH', 'H', 'NH', 'H',
       'H', 'H', 'NH', 'NH', 'H', 'NH', 'H', 'NH', 'H', 'NH', 'NH', 'H',
       'NH', 'NH', 'H', 'H', 'NH', 'H', 'H', 'H', 'H', 'NH', 'NH', 'H',
       'NH', 'H', 'NH', 'NH', 'H', 'H', 'H', 'NH', 'H', 'H', 'NH', 'NH',
       'H', 'NH', 'NH', 'H', 'NH', 'NH', 'NH', 'H', 'NH', 'H', 'NH', 'NH',
       'NH', 'H', 'H', 'H', 'H', 'NH', 'H', 'NH', 'NH', 'H', 'NH', 'NH',
       'NH', 'H', 'H', 'H', 'H', 'H', 'H', 'NH', 'NH', 'NH', 'NH', 'NH',
       'NH', 'H', 'NH', 'NH', 'NH', 'H', 'H', 'H', 'NH', 'H', 'H', 'H',
       'NH', 'NH', 'NH', 'NH', 'H', 'NH', 'NH', 'H', 'NH', 'NH', 'NH',
       'NH', 'H', 'NH', 'H', 'H', 'NH', 'NH', 'NH', 'H', 'H', 'NH', 'NH',
       'H', 'NH', 'NH', 'NH', 'NH', 'NH', 'NH', 'NH', 'H', 'NH', 'NH',
       'H', 'H', 'H', 'NH', 'NH', 'H', 'H', 'NH', 'NH', 'NH', 'NH', 'NH',
       'H', 'H', 'NH', 'NH', 'NH', 'H', 'NH', 'H', 'N

In [17]:
np.array(test_y)

array(['NH', 'H', 'H', 'NH', 'NH', 'NH', 'NH', 'NH', 'NH', 'NH', 'NH',
       'NH', 'NH', 'NH', 'H', 'H', 'H', 'H', 'NH', 'NH', 'H', 'H', 'NH',
       'H', 'NH', 'NH', 'H', 'NH', 'H', 'NH', 'H', 'NH', 'NH', 'NH', 'NH',
       'NH', 'NH', 'NH', 'NH', 'H', 'NH', 'H', 'NH', 'H', 'NH', 'NH', 'H',
       'H', 'NH', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'NH', 'H', 'NH',
       'NH', 'H', 'H', 'NH', 'NH', 'H', 'H', 'H', 'NH', 'NH', 'H', 'NH',
       'NH', 'H', 'H', 'H', 'NH', 'H', 'NH', 'H', 'NH', 'NH', 'NH', 'NH',
       'NH', 'NH', 'H', 'NH', 'H', 'H', 'H', 'NH', 'NH', 'H', 'NH', 'H',
       'NH', 'H', 'H', 'NH', 'NH', 'H', 'H', 'H', 'H', 'H', 'NH', 'H',
       'H', 'H', 'NH', 'H', 'NH', 'H', 'NH', 'H', 'H', 'NH', 'NH', 'H',
       'H', 'NH', 'NH', 'H', 'H', 'H', 'H', 'H', 'NH', 'H', 'NH', 'NH',
       'H', 'H', 'NH', 'NH', 'H', 'H', 'NH', 'H', 'H', 'NH', 'NH', 'H',
       'H', 'H', 'H', 'H', 'H', 'H', 'NH', 'NH', 'NH', 'NH', 'H', 'H',
       'NH', 'NH', 'H', 'H', 'H', 'H', 'H', 'NH', 'H', 'NH